In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from requests.exceptions import HTTPError


In [12]:
#Funkcja do pobierania wszystkich ofert ze stron

def get_offer_links(url):
    
    is_not_empt_page = True
    page_number = 135
    offer_list = []
    while is_not_empt_page: 
        param = {'page' : page_number}
        try:
            dwl_page = requests.get(url = url , params = param)
            #funkcja nie zwraca niczego jeśli poprawnie wykonała zapytanie 
            dwl_page.raise_for_status()
        except HTTPError as httperror:
            is_not_empt_page = False
            print(f'Nie odnaleziono strony {page_number}.  Blad: {httperror}')
        
        ##pobieranie adresów linków do sprawdzenia:
        soup = BeautifulSoup(dwl_page.text , 'lxml')
        body = soup.body
        div = body.find_all('a', {'class': 'teaser__anchor'})
        
        for i in div:
            offer_list.append(i.get('href'))
        page_number += 1
        #print(page_number)
    
    return offer_list
        
        

In [13]:
lista_ofert_gr = get_offer_links('https://gratka.pl/nieruchomosci/mieszkania/poznan/sprzedaz')

Nie odnaleziono strony 147.  Blad: 404 Client Error: Not Found for url: https://gratka.pl/nieruchomosci/mieszkania/poznan/sprzedaz?page=147


In [23]:
def get_offer_attributes(url):
    offer_page = requests.get(url = url)
    soup = BeautifulSoup(offer_page.text, 'lxml')
    body = soup.body
    
    #params loop
    pars_atrr = body.find('ul', {'class': 'parameters__rolled'})
    #getting list of params 
    params_list = [i.get_text() for i in pars_atrr.find_all('span')]
    params_list.remove('tel.')
    #getting list of values
    values_list = [i.get_text() for i in pars_atrr.find_all('b', class_ = 'parameters__value')]

    #stworzyłem słownik do przechowywania parametów pobranych ze strony 
    dict_object = {key : value for key, value in zip(params_list, values_list)}
    #pobieram cene mieszkania
    dict_object['price'] = body.find('span', class_ = 'priceInfo__value').get_text()
    #pobieram tytuł ogłoszenia
    dict_object['title'] = body.find('h1', class_ = 'sticker__title').get_text()
    dict_object['link'] = url
    
    return dict_object

In [60]:
#funkcja do pobieranie danych z ofert 
def download_data(lista):
    count = 0
    dict_offers = {}
    for i in lista:
        dict_offers[count] = get_offer_attributes(i)
        count += 1
    
    return dict_offers


In [63]:
data_dict = download_data(lista_ofert_gr)

#  count = 0
#     dict_offers = {}
#     for i in lista_ofert_gr[15:20]:
#         dict_offers[count] = get_offer_attributes(lista)
#         count += 1 


In [67]:
data_df = pd.DataFrame(data_dict)
#data_df.transpose()

,Edukacja,Forma / wyposażenie łazienki,Forma kuchni,Forma własności,Głośność,Komunikacja,Liczba pięter w budynku,Liczba pokoi,Lokalizacja,Materiał budynku,...,"apteka, przychodnia / szpital, weterynarz","autobus, tramwaj",link,"lotnisko, autobus, tramwaj",price,"północne, wschodnie, południowe",tel.,title,"z oknem, wanna","zachodnie, południowe"
0,NaN,NaN,NaN,NaN,NaN,NaN,13,2,"\nPoznań, ...",NaN,...,NaN,NaN,https://gratka.pl/nieruchomosci/nowe-mieszkani...,NaN,\n Zapytaj o cenę\n,NaN,NaN,"Nowe mieszkanie Poznań Winogrady, ul. Serbska 15",NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,13,2,"\nPoznań, ...",NaN,...,NaN,NaN,https://gratka.pl/nieruchomosci/nowe-mieszkani...,NaN,\n Zapytaj o cenę\n,NaN,NaN,"Nowe mieszkanie Poznań Winogrady, ul. Serbska 15",NaN,NaN
2,NaN,NaN,w aneksie,NaN,NaN,NaN,13,3,"\nPoznań, ...",NaN,...,NaN,NaN,https://gratka.pl/nieruchomosci/nowe-mieszkani...,NaN,\n Zapytaj o cenę\n,NaN,NaN,"Nowe mieszkanie Poznań Winogrady, ul. Serbska 15",NaN,NaN
3,NaN,NaN,w aneksie,NaN,NaN,NaN,13,2,"\nPoznań, ...",NaN,...,NaN,NaN,https://gratka.pl/nieruchomosci/nowe-mieszkani...,NaN,\n Zapytaj o cenę\n,NaN,NaN,"Nowe mieszkanie Poznań Winogrady, ul. Serbska 15",NaN,NaN
4,NaN,NaN,w aneksie,NaN,NaN,NaN,13,2,"\nPoznań, ...",NaN,...,NaN,NaN,https://gratka.pl/nieruchomosci/nowe-mieszkani...,NaN,\n Zapytaj o cenę\n,NaN,NaN,"Nowe mieszkanie Poznań Winogrady, ul. Serbska 15",NaN,NaN
5,NaN,NaN,w aneksie,NaN,NaN,NaN,13,3,"\nPoznań, ...",NaN,...,NaN,NaN,https://gratka.pl/nieruchomosci/nowe-mieszkani...,NaN,\n Zapytaj o cenę\n,NaN,NaN,"Nowe mieszkanie Poznań Winogrady, ul. Serbska 15",NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,13,2,"\nPoznań, ...",NaN,...,NaN,NaN,https://gratka.pl/nieruchomosci/nowe-mieszkani...,NaN,\n Zapytaj o cenę\n,NaN,NaN,"Nowe mieszkanie Poznań Winogrady, ul. Serbska 15",NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,13,2,"\nPoznań, ...",NaN,...,NaN,NaN,https://gratka.pl/nieruchomosci/nowe-mieszkani...,NaN,\n Zapytaj o cenę\n,NaN,NaN,"Nowe mieszkanie Poznań Winogrady, ul. Serbska 15",NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,13,2,"\nPoznań, ...",NaN,...,NaN,NaN,https://gratka.pl/nieruchomosci/nowe-mieszkani...,NaN,\n Zapytaj o cenę\n,NaN,NaN,"Nowe mieszkanie Poznań Winogrady, ul. Serbska 15",NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,13,2,"\nPoznań, ...",NaN,...,NaN,NaN,https://gratka.pl/nieruchomosci/nowe-mieszkani...,NaN,\n Zapytaj o cenę\n,NaN,NaN,"Nowe mieszkanie Poznań Winogrady, ul. Serbska 15",NaN,NaN


In [68]:
data_dict[17]

{'tel.': '\nPoznań,                                         Grunwald,                                         wielkopolskie ',
 'Lokalizacja': '480,00 zł',
 'Opłaty (czynsz administracyjny, media)': '51 m2',
 'Powierzchnia w m2': '2',
 'Liczba pokoi': '12',
 'Piętro': 'własność',
 'Forma własności': 'wysoki standard',
 'Stan': '2017',
 'Rok budowy': 'blok',
 'Typ zabudowy': '16',
 'Liczba pięter w budynku': 'w garażu',
 'Miejsce parkingowe': 'nowa',
 'Stan instalacji': 'oddzielna',
 'Forma kuchni': 'bardzo dobry',
 'Stan łazienki': 'ciche',
 'Głośność': 'gratka-BUL1',
 'Numer referencyjny': 'prysznic',
 'Forma / wyposażenie łazienki': 'przychodnia / szpital',
 'price': '\n          515 000,00\n      \n        zł\n      \n',
 'title': 'Mieszkanie Poznań Grunwald, ul. Bułgarska 59',
 'link': 'https://gratka.pl/nieruchomosci/mieszkanie-poznan-grunwald-ul-bulgarska-59/ob/11770245'}

In [ ]:
##usunąć dwa telefony podczas pobierania
https://gratka.pl/nieruchomosci/mieszkanie-poznan-grunwald-ul-bulgarska-59/ob/11770245

In [4]:
#liczba podstron z ofertami
start_pg_nr = int(body.find('input', {'class': 'pagination__input'}).get('min'))
end_pg_nr = int(body.find('input', {'class': 'pagination__input'}).get('max'))

In [26]:
## Dobry materiał o słownikach 
#https://realpython.com/iterate-through-dictionary-python/77

##Materiała o beautifulsoup
https://www.dataquest.io/blog/web-scraping-tutorial-python/
    
##Przetwarzeanie 
##https://thispointer.com/python-pandas-how-to-create-dataframe-from-dictionary/

https://realpython.com/python-requests/

https://thispointer.com/python-pandas-how-to-create-dataframe-from-dictionary/

In [26]:
studentAgeData = {
    'Jack' : 12,
    'Roma' : 13,
    'Ritika' : 10,
    'Aadi' : 11
}

In [31]:
pd.DataFrame(list(studentAgeData.items()))

,0,1
0,Jack,12
1,Roma,13
2,Ritika,10
3,Aadi,11
